# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
# your code here
import pandas as pd

# Load the dataset
df = pd.read_csv("sentiment140.csv", header=None, encoding="ISO-8859-1")

# Define column names
df.columns = ['target','id','date','flag','user','text']

# Sample a subset of the data (e.g., 20k records)
subset_df = df.sample(n=20000, random_state=42)

# Display the first few rows of the subset dataframe
subset_df.head()


,target,id,date,flag,user,text
541200,0,2200003196,Tue Jun 16 18:18:12 PDT 2009,NO_QUERY,LaLaLindsey0609,@chrishasboobs AHHH I HOPE YOUR OK!!!
750,0,1467998485,Mon Apr 06 23:11:14 PDT 2009,NO_QUERY,sexygrneyes,"@misstoriblack cool , i have no tweet apps fo..."
766711,0,2300048954,Tue Jun 23 13:40:11 PDT 2009,NO_QUERY,sammydearr,@TiannaChaos i know just family drama. its la...
285055,0,1993474027,Mon Jun 01 10:26:07 PDT 2009,NO_QUERY,Lamb_Leanne,School email won't open and I have geography ...
705995,0,2256550904,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,yogicerdito,upper airways problem


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [3]:
# your code here
import re
def clean_up(s):
    """
    Cleans up numbers, URLs, and special characters from a string.

    Args:
        s: The string to be cleaned up.

    Returns:
        A string that has been cleaned up.
    """
        # Remove URLs
    s = re.sub(r'http\S+', '', s)
    
    # Remove special characters and numbers
    s = re.sub(r'[^A-Za-z\s]+', ' ', s)
    
    return s

# Test the function
test_string = "@Ironhack's-#Q website 776-is http://ironhack.com [(2018)]"
cleaned_string = clean_up(test_string)
print(cleaned_string)

from nltk.tokenize import word_tokenize
def tokenize(s):
    """
    Tokenize a string.

    Args:
        s: String to be tokenized.

    Returns:
        A list of words as the result of tokenization.
    """
    tokens = word_tokenize(s)
    # Convert each token to lowercase
    tokens = [token.lower() for token in tokens]
    return tokens

# Test the functions
test_string = "@Ironhack's-#Q website 776-is http://ironhack.com [(2018)]"
cleaned_string = clean_up(test_string)
tokenized_string = tokenize(cleaned_string)
print(tokenized_string)


from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet

def stem_and_lemmatize(l):
    """
    Perform stemming and lemmatization on a list of words.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after being stemmed and lemmatized.
    """
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    stemmed_and_lemmatized = []
    for word in l:
        # Stemming
        stemmed_word = stemmer.stem(word)
        # Lemmatization without specifying POS
        lemmatized_word = lemmatizer.lemmatize(word)
        stemmed_and_lemmatized.append(stemmed_word)
        stemmed_and_lemmatized.append(lemmatized_word)
    return stemmed_and_lemmatized

# Test the function
stemmed_and_lemmatized_words = stem_and_lemmatize(tokenized_string)
print(stemmed_and_lemmatized_words)



from nltk.corpus import stopwords

def remove_stopwords(l):
    """
    Remove English stopwords from a list of strings.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after stop words are removed.
    """
    # Get English stopwords
    stop_words = set(stopwords.words('english'))
    
    # Remove stopwords
    filtered_words = [word for word in l if word not in stop_words]
    
    return filtered_words

# Test the function
stopwords_removed_words = remove_stopwords(stemmed_and_lemmatized_words)
print(stopwords_removed_words)




# Apply clean_up() function
subset_df['text_processed'] = subset_df['text'].apply(clean_up)

# Apply tokenize() function
subset_df['text_processed'] = subset_df['text_processed'].apply(tokenize)

# Apply stem_and_lemmatize() function
subset_df['text_processed'] = subset_df['text_processed'].apply(stem_and_lemmatize)

# Apply remove_stopwords() function
subset_df['text_processed'] = subset_df['text_processed'].apply(remove_stopwords)

# Display the first few rows of the dataframe
subset_df.head()





 Ironhack s Q website  is   
['ironhack', 's', 'q', 'website', 'is']
['ironhack', 'ironhack', 's', 's', 'q', 'q', 'websit', 'website', 'is', 'is']
['ironhack', 'ironhack', 'q', 'q', 'websit', 'website']


,target,id,date,flag,user,text,text_processed
541200,0,2200003196,Tue Jun 16 18:18:12 PDT 2009,NO_QUERY,LaLaLindsey0609,@chrishasboobs AHHH I HOPE YOUR OK!!!,"[chrishasboob, chrishasboobs, ahhh, ahhh, hope..."
750,0,1467998485,Mon Apr 06 23:11:14 PDT 2009,NO_QUERY,sexygrneyes,"@misstoriblack cool , i have no tweet apps fo...","[misstoriblack, misstoriblack, cool, cool, twe..."
766711,0,2300048954,Tue Jun 23 13:40:11 PDT 2009,NO_QUERY,sammydearr,@TiannaChaos i know just family drama. its la...,"[tiannachao, tiannachaos, know, know, famili, ..."
285055,0,1993474027,Mon Jun 01 10:26:07 PDT 2009,NO_QUERY,Lamb_Leanne,School email won't open and I have geography ...,"[school, school, email, email, open, open, geo..."
705995,0,2256550904,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,yogicerdito,upper airways problem,"[upper, upper, airway, airway, problem, problem]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [4]:
# your code here

from nltk.probability import FreqDist

# Combine all words in text_processed into a single list
all_words = [word for sublist in subset_df['text_processed'] for word in sublist]

# Calculate frequency distribution of all words
freq_dist = FreqDist(all_words)

# Select the top 5,000 words
top_words = freq_dist.most_common(5000)

# Display the first few top words
top_words[:10]


[('wa', 2726),
 ('go', 2669),
 ('day', 2656),
 ('get', 2427),
 ('good', 2318),
 ('like', 2043),
 ('work', 1973),
 ('quot', 1951),
 ('love', 1880),
 ('today', 1742)]

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [ ]:
# your code here

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [5]:
# your code here

# Define a function to create features for each tweet
def create_features(tweet_words, top_words):
    """
    Create features for each tweet.

    Args:
        tweet_words: List of words in the tweet.
        top_words: List of top words based on frequency.

    Returns:
        Dictionary of features for the tweet.
    """
    features = {}
    for word in top_words:
        features[word] = (word in tweet_words)
    return features

# Create the feature set
feature_set = []
for index, row in subset_df.iterrows():
    tweet_words = row['text_processed']
    is_positive = row['target'] == 4
    features = create_features(tweet_words, [word[0] for word in top_words])
    feature_set.append((features, is_positive))

# Display the first few feature sets
feature_set[:5]


[({'wa': False,
   'go': False,
   'day': False,
   'get': False,
   'good': False,
   'like': False,
   'work': False,
   'quot': False,
   'love': False,
   'today': False,
   'got': False,
   'time': False,
   'lol': False,
   'u': False,
   'one': False,
   'know': False,
   'want': False,
   'back': False,
   'amp': False,
   'see': False,
   'im': False,
   'miss': False,
   'well': False,
   'think': False,
   'thi': False,
   'still': False,
   'night': False,
   'feel': False,
   'oh': False,
   'hope': True,
   'na': False,
   'home': False,
   'need': False,
   'new': False,
   'make': False,
   'thank': False,
   'ha': False,
   'last': False,
   'much': False,
   'come': False,
   'twitter': False,
   'great': False,
   'look': False,
   'watch': False,
   'going': False,
   'wish': False,
   'tomorrow': False,
   'sleep': False,
   'haha': False,
   'sad': False,
   'week': False,
   'bad': False,
   'would': False,
   'right': False,
   'wait': False,
   'say': False,
  

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [6]:
# your code here
import nltk

# Split the feature set into training and testing sets
train_set, test_set = feature_set[:15000], feature_set[15000:]

# Train the Naive Bayes classifier
classifier = nltk.NaiveBayesClassifier.train(train_set)

# Test the classifier and calculate accuracy
accuracy = nltk.classify.accuracy(classifier, test_set)

# Print the accuracy score
print("Accuracy:", accuracy)


Accuracy: 0.727


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [8]:
# your code here
# Experiment with different numbers of top features
top_words_10000 = freq_dist.most_common(10000)
feature_set_10000 = []
for index, row in subset_df.iterrows():
    tweet_words = row['text_processed']
    is_positive = row['target'] == 4
    features = create_features(tweet_words, [word[0] for word in top_words_10000])
    feature_set_10000.append((features, is_positive))

# Split the feature set into training and testing sets
train_set_10000, test_set_10000 = feature_set_10000[:15000], feature_set_10000[15000:]

# Train the Naive Bayes classifier with 10,000 top features
classifier_10000 = nltk.NaiveBayesClassifier.train(train_set_10000)

# Test the classifier with 10,000 top features and calculate accuracy
accuracy_10000 = nltk.classify.accuracy(classifier_10000, test_set_10000)

# Print the accuracy score with 10,000 top features
print("Accuracy with 10,000 top features:", accuracy_10000)


Accuracy with 10,000 top features: 0.725


## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [7]:
# your code here

import pandas as pd
import nltk
from nltk.probability import FreqDist

def clean_up(s):
    """
    Cleans up numbers, URLs, and special characters from a string.

    Args:
        s: The string to be cleaned up.

    Returns:
        A string that has been cleaned up.
    """
    # Remove URLs
    s = re.sub(r'http\S+', '', s)
    
    # Remove special characters and numbers
    s = re.sub(r'[^A-Za-z\s]+', ' ', s)
    
    # Convert to lowercase
    s = s.lower()
    
    return s

def tokenize(s):
    """
    Tokenize a string.

    Args:
        s: String to be tokenized.

    Returns:
        A list of words as the result of tokenization.
    """
    return word_tokenize(s)

def stem_and_lemmatize(l):
    """
    Perform stemming and lemmatization on a list of words.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after being stemmed and lemmatized.
    """
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    stemmed_and_lemmatized = []
    for word in l:
        # Stemming
        stemmed_word = stemmer.stem(word)
        # Lemmatization without specifying POS
        lemmatized_word = lemmatizer.lemmatize(word)
        stemmed_and_lemmatized.append(stemmed_word)
        stemmed_and_lemmatized.append(lemmatized_word)
    return stemmed_and_lemmatized

def remove_stopwords(l):
    """
    Remove English stopwords from a list of strings.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after stop words are removed.
    """
    # Get English stopwords
    stop_words = set(stopwords.words('english'))
    
    # Remove stopwords
    filtered_words = [word for word in l if word not in stop_words]
    
    return filtered_words

def create_features(tweet_words, top_words):
    """
    Create features for each tweet.

    Args:
        tweet_words: List of words in the tweet.
        top_words: List of top words based on frequency.

    Returns:
        Dictionary of features for the tweet.
    """
    features = {}
    for word in top_words:
        features[word] = (word in tweet_words)
    return features

def train_classifier(dataset_location):
    """
    Train a Naive Bayes classifier for sentiment analysis.

    Args:
        dataset_location: Location of the dataset file.

    Returns:
        Trained Naive Bayes classifier.
    """
    # Load the dataset
    df = pd.read_csv(dataset_location, header=None, encoding="ISO-8859-1")
    df.columns = ['target','id','date','flag','user','text']
    
    # Sample a subset of the data
    subset_df = df.sample(n=20000, random_state=42)
    
    # Apply data preprocessing steps
    subset_df['text_processed'] = subset_df['text'].apply(clean_up)
    subset_df['text_processed'] = subset_df['text_processed'].apply(tokenize)
    subset_df['text_processed'] = subset_df['text_processed'].apply(stem_and_lemmatize)
    subset_df['text_processed'] = subset_df['text_processed'].apply(remove_stopwords)
    
    # Calculate frequency distribution of all words
    all_words = [word for sublist in subset_df['text_processed'] for word in sublist]
    freq_dist = FreqDist(all_words)
    top_words = freq_dist.most_common(5000)
    
    # Create the feature set
    feature_set = []
    for index, row in subset_df.iterrows():
        tweet_words = row['text_processed']
        is_positive = row['target'] == 4
        features = create_features(tweet_words, [word[0] for word in top_words])
        feature_set.append((features, is_positive))
    
    # Split the feature set into training and testing sets
    train_set, test_set = feature_set[:15000], feature_set[15000:]
    
    # Train the Naive Bayes classifier
    classifier = nltk.NaiveBayesClassifier.train(train_set)
    
    return classifier


## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [9]:
# your code here

# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer
from pyspark.ml import Pipeline
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Sentiment Analysis") \
    .getOrCreate()

# Load the dataset
df = spark.read.csv("dbfs:/path/to/dataset.csv", header=True, inferSchema=True)

# Data preprocessing
# Clean up data, tokenize text, remove stop words
tokenizer = Tokenizer(inputCol="text", outputCol="words")
stop_words_remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
pipeline = Pipeline(stages=[tokenizer, stop_words_remover])
cleaned_df = pipeline.fit(df).transform(df)

# Feature engineering
# Convert words into numerical features using CountVectorizer
count_vectorizer = CountVectorizer(inputCol="filtered_words", outputCol="features")
count_vectorizer_model = count_vectorizer.fit(cleaned_df)
features_df = count_vectorizer_model.transform(cleaned_df)

# Split the data into training and testing sets
train_set, test_set = features_df.randomSplit([0.75, 0.25], seed=42)

# Train the Naive Bayes classifier
nb = NaiveBayes(labelCol="target", featuresCol="features")
nb_model = nb.fit(train_set)

# Make predictions on the test set
predictions = nb_model.transform(test_set)

# Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="target", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Accuracy:", accuracy)

# Stop the SparkSession
spark.stop()


ModuleNotFoundError: No module named 'pyspark'